##### Proceso de diseño de funciones

* dependencias

In [11]:
import pandas as pd
import ast

In [5]:
movies = pd.read_csv("datasets/movies_final.csv", index_col = ["id"])
movies.shape

(13051, 19)

In [14]:
cast = pd.read_csv("datasets/cast.csv", index_col = ["id"])
cast.shape

(206158, 3)

In [76]:
directors = pd.read_csv("datasets/directors.csv", index_col = ["id"])
directors.shape

(19895, 3)

* **cantidad de filmaciones por mes**

In [ ]:
months = ["enero","febrero","marzo","abril","mayo","junio","julio","agosto","septiembre","octubre","noviembre","dicienmbre"]
def cantidad_filmaciones_mes( Mes : str): #Se ingresa un mes en idioma Español. 
    index = months.index(Mes.lower()) +1
    movies_in_month = movies[pd.to_datetime(movies["release_date"]).dt.month == index]
    return f"{len(movies_in_month)} películas fueron estrenadas en el mes de {Mes.lower().capitalize()}"

In [29]:
for m in months:
    print(cantidad_filmaciones_mes(m))

1029 cantidad de películas fueron estrenadas en el mes de Enero
903 cantidad de películas fueron estrenadas en el mes de Febrero
1031 cantidad de películas fueron estrenadas en el mes de Marzo
972 cantidad de películas fueron estrenadas en el mes de Abril
1010 cantidad de películas fueron estrenadas en el mes de Mayo
970 cantidad de películas fueron estrenadas en el mes de Junio
940 cantidad de películas fueron estrenadas en el mes de Julio
1102 cantidad de películas fueron estrenadas en el mes de Agosto
1547 cantidad de películas fueron estrenadas en el mes de Septiembre
1345 cantidad de películas fueron estrenadas en el mes de Octubre
945 cantidad de películas fueron estrenadas en el mes de Noviembre
1257 cantidad de películas fueron estrenadas en el mes de Dicienmbre


* **cantidad de filmaciones por dia**

In [30]:
weekdays = ["lunes", "martes", "miércoles", "jueves", "viernes", "sábado", "domingo"]
def cantidad_filmaciones_dia( Dia :str): #Se ingresa un mes en idioma Español.
    index = weekdays.index(Dia.lower())
    movies_in_day = movies[pd.to_datetime(movies["release_date"]).dt.weekday == index]
    return f"{len(movies_in_day)} películas fueron estrenadas en los días {Dia.lower().capitalize()}"

In [31]:
for d in weekdays:
    print(cantidad_filmaciones_dia(d))

727 películas fueron estrenadas en los días Lunes
1182 películas fueron estrenadas en los días Martes
2210 películas fueron estrenadas en los días Miércoles
2276 películas fueron estrenadas en los días Jueves
4902 películas fueron estrenadas en los días Viernes
1030 películas fueron estrenadas en los días Sábado
724 películas fueron estrenadas en los días Domingo


* **popularidad de la filmación**

In [ ]:
def score_titulo( titulo_de_la_filmación ): # Se ingresa el título de una filmación esperando como respuesta el título, el año de estreno y el score.
    movie_candidates = movies.loc[movies["title"].str.lower() == titulo_de_la_filmación.lower()]
    if len(movie_candidates) > 0:
        movie = movie_candidates.iloc[0]
    else:
        movie = None
    
    if movie is None:
        return f"La filmacion {titulo_de_la_filmación} no se encuestra en la base de datos"
    return f"La película {movie['title']} fue estrenada en el año {movie['release_year']} con un score/popularidad de {movie['popularity']:.6f}"

In [49]:
score_titulo("Toy Story")

'La película Toy Story fue estrenada en el año 1995 con un score/popularidad de 21.946943'

* **cantidad de votos y promedio por voto de la filmación**

In [8]:
def votos_titulo( titulo_de_la_filmación ): # Se ingresa el título de una filmación esperando como respuesta el título, la cantidad de votos y el valor promedio de las votaciones. La misma variable deberá de contar con al menos 2000 valoraciones, caso contrario, debemos contar con un    mensaje avisando que no cumple esta condición y que por ende, no se devuelve ningun valor.
    movie_candidates = movies.loc[movies["title"].str.lower() == titulo_de_la_filmación.lower()]
    if len(movie_candidates) > 0:
        movie = movie_candidates.iloc[0]
    else:
        movie = None
    
    if movie is None:
        return f"La filmacion {titulo_de_la_filmación} no se encuestra en la base de datos"
    
    votes = movie["vote_count"]
    votes_avg = movie["vote_average"]
    msj = "Sin votos suficientes." if int(votes) < 2000 else f'La misma cuenta con un total de {int(movie["vote_count"])} valoraciones, con promedio de {votes_avg:.4f}.'
    return f'La película {movie["title"]} fue estrenada en el año {movie["release_year"]}. {str(msj)}'

In [62]:
votos_titulo("Toy Story")

'La película Toy Story fue estrenada en el año 1995. La misma cuenta con un total de 5415 valoraciones, con promedio de 7.7000.'

* **cantidad de filmaciones en las que aparece el actor y su retorno**

In [58]:

def get_actor( nombre_actor ): # Se ingresa el nombre de un actor que se encuentre dentro de un dataset debiendo devolver el éxito del mismo medido a través del retorno. Además, la cantidad de películas que en las que ha participado y el promedio de retorno. La definición no deberá considerar directores.
    actor = cast[cast["name"].str.lower() == nombre_actor.lower()]
    
    if actor.empty:
        return f'No se ha encontrado un actor llamado {nombre_actor}'
    
    actor = actor.iloc[0]
    movies_ids = ast.literal_eval(actor["performances"])
    
    ret = movies[movies.index.isin(movies_ids)]["return"].sum()
    
    perfs = len(movies_ids)
    
    return f'El actor {actor["name"]} ha participado de {perfs} cantidad de filmaciones, el mismo ha conseguido un retorno de {ret} con un promedio de {ret/perfs :.6f} por filmación'

In [60]:
get_actor("Don Rickles")

'El actor Don Rickles ha participado de 29 cantidad de filmaciones, el mismo ha conseguido un retorno de 27.800190947905985 con un promedio de 0.958627 por filmación'

* **cantidad de filmaciones en dirigidas por el director, su retorno y una descripción de cada una**

In [86]:
def get_movie_description(row):
    return f'{row["title"]} | {row["release_date"]} | retorno: {row["return"]} , costo: {row["budget"]} , ganancia: {row["revenue"]}'

In [97]:
def get_director( nombre_director ): # Se ingresa el nombre de un director que se encuentre dentro de un dataset debiendo devolver el éxito del mismo medido a través del retorno. Además, deberá devolver el nombre de cada película con la fecha de lanzamiento, retorno individual, costo y ganancia de la misma.
    direc = directors[directors["name"].str.lower() == nombre_director.lower()]
    
    if direc.empty:
        return f'No se ha encontrado un director llamado {nombre_director}'
    direc = direc.iloc[0]
    
    movies_ids = ast.literal_eval(direc["directions"])
    directed_movies = movies[movies.index.isin(movies_ids)]
    
    ret = directed_movies["return"].sum()
    dirs = len(directed_movies)
    
    movies_desc = ""
    for _, m in directed_movies.iterrows():
        movies_desc += get_movie_description(m) + "\n"
    
    return f'El actor {direc["name"]} ha participado de {dirs} cantidad de filmaciones, el mismo ha conseguido un retorno de {ret} con un promedio de {ret/dirs :.6f} por filmación.\n {movies_desc}'

In [98]:
get_director("John Lasseter")

"El actor John Lasseter ha participado de 10 cantidad de filmaciones, el mismo ha conseguido un retorno de 27.65437835777778 con un promedio de 2.765438 por filmación.\n Toy Story | 1995-10-30 | retorno: 12.4518011 , costo: 30000000 , ganancia: 373554033.0\nA Bug's Life | 1998-11-25 | retorno: 3.027157158333333 , costo: 120000000 , ganancia: 363258859.0\nToy Story 2 | 1999-10-30 | retorno: 5.526298544444445 , costo: 90000000 , ganancia: 497366869.0\nLuxo Jr. | 1986-08-17 | retorno: 0.0 , costo: 0 , ganancia: 0.0\nCars | 2006-06-08 | retorno: 3.849859575 , costo: 120000000 , ganancia: 461983149.0\nCars 2 | 2011-06-11 | retorno: 2.79926198 , costo: 200000000 , ganancia: 559852396.0\nTin Toy | 1988-08-01 | retorno: 0.0 , costo: 0 , ganancia: 0.0\nRed's Dream | 1987-08-17 | retorno: 0.0 , costo: 0 , ganancia: 0.0\nKnick Knack | 1989-01-01 | retorno: 0.0 , costo: 0 , ganancia: 0.0\nMater and the Ghostlight | 2006-07-27 | retorno: 0.0 , costo: 0 , ganancia: 0.0\n"